# Reinforcement Learning

## Discrete control problems - Q-learning
Q-learning is the fundamental algorithm of RL that maintains so called Q-table $Q(s,a)$ of total reward when in the state $s$ and action $a$ is taken. Q-table is updated using either random exploration (random actions) or $\epsilon$-greedy algorithm. The update rule is
$$
Q(S_t,A_t) \leftarrow Q(S_t,A_t)+\alpha\left( R_{t+1}+\gamma \max_a Q(S_{t+1},a)-Q(S_t,A_t)\right) ,
$$
where $\alpha \in ]0,1]$ is the learning rate (how much new observations are used) and $\gamma \in [0,1[$ is reward delay. 

### Demo 1: Frozen lake

Frozen lake has many uncertainties. For example, when you try to move (left, right, up or down) you sometimes do not move, you sometimes move to wrong direction. Moreover, in some locations the ice is too thin and you fall to cold water (game over). Can you learn to find your way over the frozen lake? You need to particularly understand what happens with learning when you move from non-slippery ice to slippery ice.

In [ ]:
import gym
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Create environment
env = gym.make("FrozenLake-v1", is_slippery=False)
env.reset()
env.render()

In [ ]:
action_size = env.action_space.n
print("Action size: ", action_size)

state_size = env.observation_space.n
print("State size: ", state_size)

In [ ]:
done = False
env.reset()
while not done:
    #action = np.random.randint(0,4) # 0:Left 1:Down 2: Right, 3: Up
    action = int(input('0/left 1/down 2/right 3/up:'))
    new_state, reward, done, info = env.step(action)
    time.sleep(1.0) 
    print(f'S_t+1={new_state}, R_t+1={reward}, done={done}')
    env.render()

Let's initialize Q-table structure

In [ ]:
qtable_history = []
score_history = []
qtable = np.zeros((state_size, action_size))

total_episodes = 10000        # Total episodes
learning_rate = 1.0          # Learning rate alpha
max_steps = 100              # Max steps per episode
gamma = 0.9                  # Discounting rate

# Exploration parameters (not really needed)
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.001            # Minimum exploration probability 
decay_rate = 0.00005             # Exponential decay rate for exploration prob

Let's define an evaluation function that runs the current policy (action that maximises Q-value in each state) and return average reward

In [ ]:
def eval_policy(qtable_, num_of_episodes_, max_steps_):
    env.reset()
    total_test_episodes = 1000
    rewards = []

    for episode in range(num_of_episodes_):
        state = env.reset()
        step = 0
        done = False
        total_rewards = 0

        for step in range(max_steps_):
            action = np.argmax(qtable_[state,:])
            new_state, reward, done, info = env.step(action)
            total_rewards += reward
        
            if done:
                rewards.append(total_rewards)
                break
            state = new_state
    env.close()
    avg_reward = sum(rewards)/num_of_episodes_
    return avg_reward

Let's test the initial Q-table full of zeros. Note that you need to define number of episodes and max number of steps so that vaues stabilize over random runs.

In [ ]:
print(eval_policy(qtable,1000,100))

Let's make optimal Q-table manually and test average reward it achieves

In [ ]:
qtable_opt = np.zeros((state_size, action_size))
qtable_opt[0,:] = [0,0,1,0] # Row 1
qtable_opt[1,:] = [0,0,1,0] # 
qtable_opt[2,:] = [0,1,0,0] # 
qtable_opt[3,:] = [1,0,0,0] # 
qtable_opt[4,:] = [0,1,0,0] # Row 2
qtable_opt[5,:] = [0,1,0,0] # 
qtable_opt[6,:] = [0,1,0,0] # 
qtable_opt[7,:] = [1,0,0,0] # 
qtable_opt[8,:] = [0,0,1,0] # Row 3
qtable_opt[9,:] = [0,0,1,0] # 
qtable_opt[10,:] = [0,1,0,0] # 
qtable_opt[11,:] = [0,1,0,0] # 
qtable_opt[12,:] = [0,0,1,0] # Row 3
qtable_opt[13,:] = [0,0,1,0] # 
qtable_opt[14,:] = [0,0,1,0] # 
qtable_opt[15,:] = [0,0,0,0] # 
print(eval_policy(qtable_opt,1000,100))

OK. let's try to learn the optimal Q-table using the Q-learning update rule. Particularly we should study optimal actions near the goal to make sure the system converged to correct solution.

In [ ]:
# List of rewards
rewards = []

episode_count = 0
# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = np.random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration random integer in [0,3]
        else:
            #action = env.action_space.sample() # OpenAI Gym provides this
            action = np.random.randint(0,4)

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    #epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)
    
    episode_count = episode + 1
    if episode_count % 1000 == 0 or episode_count == 1:
        print(eval_policy(qtable,1000,100))
        #qtable_history.append(qtable)
        #score_history.append(sum(rewards)/episode_count)
        #save_canvas(qtable, 800, 800, filename = "./output/FrozenLake_ep" + str(episode_count) + ".png")

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)
env.reset()
env.render()

## Continuous control problems
Discrete problems mainly appear in games and other virtual environments, but robot measurements are continuous (continuous states) and often also control signals are continuous. It is important to understand what works there.

### Demo 2: CartPole controller

Let's load the necessary packages

In [ ]:
import numpy as np # used for arrays
import gym # pull the environment
import time # to get the time
import math # needed for calculations
import time

CartPole provides observations of the cart location in $[-4.8,+4.8]$ units, cart acceleration in $[-\infty,+\infty]$, pole angle in radians in $[-24^\circ,+24^\circ]$ and pole acceleration in in $[-\infty,+\infty]$. The control signal is a discrete "push" of the cart to the left, $a=0$, or right, $a=1$.

In [ ]:
env = gym.make("CartPole-v1")
print(env.observation_space.low)
print(env.observation_space.high)
print(env.action_space)

Let's have an example run.

In [ ]:
env.reset()
done = False
while not done:
    action = np.random.randint(0, 2) # Random action
    #action = 1 # Fixed action, 0 or 1
    new_state, reward, done, _ = env.step(action)
    print(new_state)
    print(reward)
    print(done)
    env.render()
    time.sleep(0.1)
env.close()

Q-learning needs discrete values so let's discretize the space observation space. Number of bins is important variable here. For example, 20 bins may converge well while 10 almost never.

In [ ]:
np_array_window_size = np.array([0.25, 0.25, 0.01, 0.1])
num_of_bins = 40
cart_loc_bins = np.linspace(env.observation_space.low[0],env.observation_space.high[0],num_of_bins)
cart_acc_bins = np.linspace(-10,+10,num_of_bins)
pole_angle_bins = np.linspace(env.observation_space.low[2],env.observation_space.high[2],num_of_bins)
pole_acc_bins = np.linspace(-10,+10,num_of_bins)
def get_discrete_state(state):
    cart_loc = np.argmin(np.abs(cart_loc_bins-state[0]))
    cart_acc = np.argmin(np.abs(cart_acc_bins-state[1]))
    pole_ang = np.argmin(np.abs(pole_angle_bins-state[2]))
    pole_acc = np.argmin(np.abs(pole_acc_bins-state[3]))
    return tuple([cart_loc, cart_acc, pole_ang, pole_acc])
                         
    #discrete_state = state/np_array_window_size+ np.array([15,10,1,10])
    #return tuple(discrete_state.astype(np.int))

Let's re-run with discrete states. By running the code with actions 0 and 1 several times, we found that during the valid episode step (done is False) the indeces run from -14..14, -14..14, -19..+19, 

In [ ]:
env.reset()
done = False
while not done:
    action = np.random.randint(0, 2)
    #action = 1
    new_state, reward, done, _ = env.step(action)
    print(get_discrete_state(new_state))
    print(reward)
    print(done)
    env.render()
    time.sleep(0.1)
env.close()

In [ ]:
LEARNING_RATE = 0.1

DISCOUNT = 0.95
EPISODES = 40000
total = 0
total_reward = 0
prior_reward = 0

Observation = [30, 30, 50, 50]


epsilon = 1

epsilon_decay_value = 0.99995

In [ ]:
#q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))

q_table = np.random.uniform(low=0, high=1, size=(num_of_bins, num_of_bins, num_of_bins, num_of_bins, env.action_space.n))

q_table.shape

In [ ]:

for episode in range(EPISODES + 1): # go through the episodes
    t0 = time.time() # set the initial time
    discrete_state = get_discrete_state(env.reset()) # get the discrete start for the restarted environment 
    done = False
    episode_reward = 0 # reward starts as 0 for each episode

    if episode % 2000 == 0: 
        print("Episode: " + str(episode))

    while not done: 

        if np.random.random() > epsilon:
            action = np.argmax(q_table[discrete_state]) # take cordinated action
        else:

            action = np.random.randint(0, env.action_space.n) # do a random ation

        #c = env.step(action) # step action to get new states, reward, and the "done" status.
        new_state, reward, done, _ = env.step(action)
        episode_reward += reward # add the reward

        new_discrete_state = get_discrete_state(new_state)

        if episode % 2000 == 0: # render
            env.render()

        if not done: # update q-table
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state

    if epsilon > 0.05: # epsilon modification
        if episode_reward > prior_reward and episode > 10000:
            epsilon = math.pow(epsilon_decay_value, episode - 10000)
    
            if episode % 500 == 0:
                print("Epsilon: " + str(epsilon))

    t1 = time.time() # episode has finished
    episode_total = t1 - t0 # episode total time
    total = total + episode_total

    total_reward += episode_reward # episode total reward
    prior_reward = episode_reward

    if episode % 1000 == 0: # every 1000 episodes print the average time and the average reward
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0

env.close()

Let's test how well it works

In [ ]:
new_state = env.reset()
print(new_state)
for s in range(0,100):
    #action = np.random.randint(0, 2)
    discrete_state = get_discrete_state(new_state)
    action = np.argmax(q_table[discrete_state])
    new_state, reward, done, _ = env.step(action)
    print(get_discrete_state(new_state))
    print(reward)
    print(done)
    env.render()
    time.sleep(0.1)
env.close()

It might be useful to think why epsilon sampling (with low value) might still be useful when the controller (policy) is used in real case?

### Demo 3: CartPole using Deep Q-Learning (DQN)

In this demo we use the Deep Q-Learning Network proposed by Mnih et al. in 2013. Their idea allowed the fundamental algorithms that helped computers to learn play computer games just by playing them and collecting experiences to train a deep network policy. In their case input is the current image of game that cannot anymore be discretized but needs to be treated as continuous state. In this case $Q(s,a)$ depends on continuous $D$-dimensional state $s \in \mathbb{R}^D$ and discrete actions $a = 0, 1, 2, \ldots$ that can be encoded as one-hot input.

See https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

In [ ]:
import numpy as np # used for arrays
import gym # pull the environment
import time # to get the time
import math # needed for calculations
import time

In [ ]:
env = gym.make("CartPole-v1")
print(env.observation_space.low)
print(env.observation_space.high)
print(env.action_space)

In [ ]:
#initializer = tf.keras.initializers.Zeros()
#
#model = tf.keras.models.Sequential([
#  tf.keras.layers.InputLayer(input_shape=(4,)),
#  tf.keras.layers.Dense(32, activation='relu', kernel_initializer=initializer),
#  tf.keras.layers.Dense(32, activation='relu', kernel_initializer=initializer),
#  tf.keras.layers.Dense(2, activation='linear', kernel_initializer=initializer)
#])

model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(4,)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(2, activation='linear')
])

In [ ]:
loss_fn = tf.keras.losses.MeanSquaredError()
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['mse'])
model.summary()

Let's test using initial network

In [ ]:
def eval_dqn(dqn_, num_of_episodes_, max_steps_):
    env.reset()
    total_test_episodes = 1000
    rewards = []

    for episode in range(num_of_episodes_):
        state = env.reset()
        step = 0
        done = False
        total_rewards = 0

        for step in range(max_steps_):
            action = np.argmax(dqn_.predict(state.reshape(1,-1)))
            new_state, reward, done, info = env.step(action)
            total_rewards += reward
        
            if done:
                break
            state = new_state
        rewards.append(total_rewards)
    env.close()
    avg_reward = sum(rewards)/num_of_episodes_
    return avg_reward

In [ ]:
print(f'Current DQN perf {eval_dqn(model,10,50)}')

In [ ]:
new_state = np.array(env.reset())
done = False
while not done:
    print(model.predict(new_state.reshape(1,-1)))
    action = np.argmax(model.predict(new_state.reshape(1,-1)))
    print(action)
    new_state, reward, done, _ = env.step(action)
    print(new_state)
    print(reward)
    print(done)
    env.render()
    time.sleep(0.1)
env.close()

DQN main loop

In [ ]:
num_of_episodes = 10000
gamma = 0.9
epsilon = 1.0
epsilon_decay_value = 0.99995

total_reward = 0
prior_reward = 0

buffer_size = 10000
tr_batch_size = 100
tr_freq = 10

state_buffer = np.zeros((buffer_size,4))
reward_buffer = np.zeros((buffer_size,1))
action_buffer = np.zeros((buffer_size,1))
done_buffer = np.zeros((buffer_size,1))
new_state_buffer = np.zeros((buffer_size,4))

buffer_count = 0
buffer_full = False
for episode in range(num_of_episodes): # go through the episodes

    state = env.reset()
    done = False
    episode_reward = 0 # reward starts as 0 for each episode

    if episode % 100 == 0: 
        print("Episode: " + str(episode))

    while not done: 
        
        # Q-net prediction
        Q_net_pred = model.predict(state.reshape(1,-1))

        # Select random or prediction        
        if np.random.random() > epsilon:
            action = np.argmax(Q_net_pred)
        else:
            action = np.random.randint(0, env.action_space.n) # do a random ation

        new_state, reward, done, _ = env.step(action)
        episode_reward += reward # add the reward
        
        # Store buffer variables
        if buffer_count < buffer_size:
            buf_ind = buffer_count
        else:
            buffer_count = 0
            buf_ind = buffer_count
            buffer_full = True
            
        state_buffer[buf_ind,:] = state
        action_buffer[buf_ind] = action
        reward_buffer[buf_ind] = reward
        new_state_buffer[buf_ind,:] = new_state
        done_buffer[buf_ind] = done
        
        # Increments
        state = new_state
        buffer_count = buffer_count+1

    if epsilon > 0.05: # epsilon modification
        if episode_reward > prior_reward and episode > 1000:
            epsilon = math.pow(epsilon_decay_value, episode - 1000)
            if episode % 500 == 0:
                print("Epsilon: " + str(epsilon))

    total_reward += episode_reward # episode total reward
    prior_reward = episode_reward

    if buffer_full and episode % tr_freq == 0: 
        # Train network
        X = np.zeros((tr_batch_size,4))
        Y = np.zeros((tr_batch_size,2))
        for ind, tr_ind in enumerate(np.random.randint(buffer_size,size=tr_batch_size)):
            X[ind,:] = state_buffer[tr_ind,:]
            Y[ind,:] = model.predict(X[ind,:].reshape(1,-1))
            if done_buffer[tr_ind]:
                Y[ind, int(action_buffer[tr_ind])] = reward
            else:                
                Y[ind, int(action_buffer[tr_ind])] = reward+gamma*np.max(model.predict(new_state_buffer[tr_ind,:].reshape(1,-1)))
        
        model.fit(X,Y,epochs=4,verbose=1)
        #model.fit(X,Y,verbose=1)
        
    if episode % 100 == 0: # every 1000 episodes print the average time and the average reward
        print(f'Current DQN perf {eval_dqn(model,10,50)}')

    if episode % 1000 == 0: # every 1000 episodes print the average time and the average reward
        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0
 
env.close()

## References

R.S. Sutton and A.G. Barto (2021): Reinforcement Learning: An Introduction. 2n ed. URL: http://incompleteideas.net/book/the-book.html 

Christopher Wong: "FrozenLake" Blog post. URL: https://cwong8.github.io/projects/FrozenLake/

Ali Fakhry (2020): "Using Q-Learning for OpenAI’s CartPole-v1" Blog Post. URL: https://medium.com/swlh/using-q-learning-for-openais-cartpole-v1-4a216ef237df

Greg Surma: "Cartpole - Introduction to Reinforcement Learning (DQN - Deep Q-Learning)" Blog post. URL: https://gsurma.medium.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288

Volodymyr Mnih, Koray Kavukcuoglu, David Silver, Alex Graves, Ioannis Antonoglou, Daan Wierstra, Martin Riedmiller (2013): "Playing Atari With Deep Reinforcement Learning", NeurIPS Deep Learning Workshop. URL: https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf 
